In [84]:
from global_funcs import load_program_config
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
import time

cluster = LocalCUDACluster()
client = Client(cluster)

import dask_cudf

2022-05-17 02:25:46,388 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


In [85]:
configs = load_program_config()

clean_fasta_filepath = configs['clean_fasta_file']
base_col_names =  configs['base_col_names']
fasta_sep = configs['fasta_sep']
label_col_name = configs['label_col_name']
label_regex = configs['label_regex']
random_seed = configs['random_seed']


In [86]:
def extract_labels(df):
    df['clean_labels'] = df[label_col_name].str.extract(label_regex).loc[:, 0]
    return df

In [87]:
df = dask_cudf.read_parquet(clean_fasta_filepath)

In [88]:
df['clean_labels'] = None

In [89]:
df = df.map_partitions(extract_labels)

In [91]:
unq_labs = df.sort_values('clean_labels')['clean_labels'].unique().compute()

In [93]:
sub_unq_labs = unq_labs.to_frame().sample(10, random_state=random_seed)['clean_labels'].to_arrow().to_pylist()

In [94]:
df = df.set_index('clean_labels', sorted=True).loc[sub_unq_labs].reset_index(drop=True)

In [97]:
out_name = clean_fasta_filepath + '_10_cats'

In [98]:
_ = df.to_parquet(out_name)

In [99]:
del df

In [100]:
client.shutdown()
client.close()